In [3]:
import pandas as pd
import yfinance
from datetime import date, timedelta, datetime
import numpy as np
import math
from tqdm import tqdm
import pickle
from sklearn.preprocessing import LabelEncoder 

## Wczytanie danych podstawowych

In [10]:
url = "https://raw.githubusercontent.com/MarylaSosna/umwf_projekt/main/dane_basic.csv"
df = pd.read_csv(url, sep=',', header = None, names = ['Date', 'Comp2','Comp','Category','Code', 'Score'])
df = df.drop('Code', axis = 1)
df = df.drop('Comp2', axis=1)
df = df.reset_index(drop = True)
df["Date"] = pd.to_datetime(df["Date"])

In [35]:
df.head

,Date,Comp,Category,Score
0,2004-02-11,SU,Energy Minerals,0.953727
1,2004-02-11,GGG,Producer Manufacturing,0.952753
2,2004-02-11,WGR,Energy Minerals,0.947634
3,2004-02-11,CWT,Utilities,0.934181
4,2004-02-11,BLL,Process Industries,0.922862
5,2004-02-11,APA,Energy Minerals,0.912117
6,2004-02-11,JW.B,Consumer Services,0.906333
7,2004-02-11,MATX,Transportation,0.866946
8,2004-02-11,ROST,Retail Trade,0.864789
9,2004-02-11,AXL,Producer Manufacturing,0.861478


In [12]:
df.Score.isna().sum()

0

In [13]:
df.shape

(37360, 4)

In [14]:
df.Comp.nunique()

1834

## Pobieranie ceny zamknięcia (Close) dla każdej firmy (Comp)

In [156]:
comp_names = set(df.Comp)

In [157]:
beginnig_date = datetime(2004, 2, 11)
end_date = datetime.now()

In [16]:
closes_df = pd.DataFrame()
# comp_names = set({"SU", "GGG", "WGR"})

for comp in comp_names:
    getInfo = yfinance.Ticker(comp)
    getHistory = getInfo.history(getInfo, start=beginnig_date, end=end_date)

    getHistory["Comp"] = comp
    getHistory["Date"] = getHistory.index
    closes_df = closes_df.append(getHistory, ignore_index=True)

closes_df = closes_df.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits", "Adj Close"])

- VIA.XX10: No data found, symbol may be delisted
- CATM: No data found, symbol may be delisted
- ASFI: No data found, symbol may be delisted
- UTIW: No data found for this date range, symbol may be delisted
- WSO.B: No data found, symbol may be delisted
- STRZB: No data found for this date range, symbol may be delisted
- PD.XX1: No data found, symbol may be delisted
- VICR: No data found for this date range, symbol may be delisted
- PPS: No data found for this date range, symbol may be delisted
- DLLR: No data found for this date range, symbol may be delisted
- PNY: No data found for this date range, symbol may be delisted
- BKC.XX10: No data found, symbol may be delisted
- WFSI: No data found for this date range, symbol may be delisted
- ROGFF: No data found, symbol may be delisted
- GEF.B: No data found for this date range, symbol may be delisted
- EC.XX9: No data found, symbol may be delisted
- SIRO: No data found for this date range, symbol may be delisted
- BGGSQ: No data found, 

- GVHR: No data found for this date range, symbol may be delisted
- LABL: No data found, symbol may be delisted
- BIO.B: No data found for this date range, symbol may be delisted
- CMD: No data found, symbol may be delisted
- LIFE.XX5: No data found, symbol may be delisted
- NBL: No data found, symbol may be delisted
- PLD.XX2: No data found, symbol may be delisted
- RAI: No data found for this date range, symbol may be delisted
- VIAC: No data found, symbol may be delisted
- DNEX: No data found for this date range, symbol may be delisted
- EV: No data found, symbol may be delisted
- HCR.XX1: No data found, symbol may be delisted
- CHSI: No data found for this date range, symbol may be delisted
- POT: No data found for this date range, symbol may be delisted
- MHM.XX1: No data found, symbol may be delisted
- CMO: No data found, symbol may be delisted
- Q.XX1: No data found, symbol may be delisted
- KDN: No data found for this date range, symbol may be delisted
- CEC: No data found for 

- ICUI: No data found for this date range, symbol may be delisted
- MKC.V: No data found, symbol may be delisted
- VARI: No data found for this date range, symbol may be delisted
- NTRI: No data found, symbol may be delisted
- CMG.B: No data found, symbol may be delisted
- CORE: No data found, symbol may be delisted
- CLGX: No data found, symbol may be delisted
- LBYYQ: No data found, symbol may be delisted
- HPOL: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- IGT.XX1: No data found, symbol may be delisted
- SII.XX1: No data found, symbol may be delisted
- GGP: No data found for this date range, symbol may be delisted
- PTRY: No data found for this date range, symbol may be delisted
- BJ.XX10: No data found, symbol may be delisted
- TSS: No data found, symbol may be delisted
- CBST: No data found for this date range, symbol may be delisted
- MRD.XX1: No data found, symbol may be delisted
- ANH: No data found, symbol may be de

In [159]:
closes_df.shape

(5599549, 3)

In [160]:
len(set(closes_df.Comp))

1411

## Wyznaczanie stopy zwrotu

Wyliczanie stopy zwrotu w 4 różnych horyzontach czasowych (miesiąc, kwartał, pół roku, rok) zgdonie ze wzorem

$r_t = ln\frac{C_t}{C_{t-1}}$

$C_{t-1}$ - cena zamknięcia (Close) w okresie poprzedzającym (miesiąc temu, kwartał temu itd.)

In [16]:
companies = set(closes_df.Comp)
df_rors_all = pd.DataFrame()

for comp in companies:
    comp_df = closes_df[closes_df.Comp == comp].copy()

    for date in comp_df.Date:
        try:
            # wartość bazowa c_{t-1}
            c_t1 = comp_df[comp_df.Date == date].iloc[0, 0]

            # wartość za miesiąc
            date_plus_month = date + pd.DateOffset(months=1)
            if date_plus_month in set(comp_df.Date):
                c_t_month = comp_df[comp_df.Date == date_plus_month].iloc[0, 0]
                ror_month = math.log(c_t_month / c_t1)
                comp_df.loc[comp_df["Date"] == date_plus_month, "ROR_month"] = ror_month
        
        except ValueError:
            ror_month = np.nan
            continue
            
        try:
            # wartość za kwartał
            date_plus_qtr = date + pd.DateOffset(months=3)
            if date_plus_qtr in set(comp_df.Date): 
                c_t_qtr = comp_df[comp_df.Date == date_plus_qtr].iloc[0, 0]
                ror_qtr = math.log(c_t_qtr / c_t1)
                comp_df.loc[comp_df["Date"] == date_plus_qtr, "ROR_qtr"] = ror_qtr
                
        except ValueError:
            ror_qtr = np.nan
            continue

        try:    
            # wartość za pół roku
            date_plus_halfy = date + pd.DateOffset(months=6)
            if date_plus_halfy in set(comp_df.Date):
                c_t_halfy = comp_df[comp_df.Date == date_plus_halfy].iloc[0, 0]
                ror_halfy = math.log(c_t_halfy / c_t1)
                comp_df.loc[comp_df["Date"] == date_plus_halfy, "ROR_half_year"] = ror_halfy
                
        except ValueError:
            ror_halfy = np.nan
            continue

        try:     
            # wartość za rok
            date_plus_year = date + pd.DateOffset(years=1)
            if date_plus_year in set(comp_df.Date):
                c_t_year = comp_df[comp_df.Date == date_plus_year].iloc[0, 0]
                ror_year = math.log(c_t_year / c_t1)
                comp_df.loc[comp_df["Date"] == date_plus_year, "ROR_year"] = ror_year
            
        except ValueError:
            ror_year = np.nan
            continue
            
    df_rors_all = df_rors_all.append(comp_df, ignore_index=True)

In [22]:
df_rors_all.shape

(5599549, 7)

In [23]:
df_rors_all.Comp.nunique()

1411

In [169]:
df_rors_all.isna().sum()

Close                 18
Comp                   0
Date                   0
ROR_month        2312063
ROR_qtr          1155117
ROR_half_year    1527215
ROR_year         1713116
dtype: int64

## Połączenie danych

Jak widać, w 5 mln zbiorze znajduje się wiele braków. Wynika to z faktu, że 1) dla niektórych spółek nie udało się pobrac danych z Yahoo Finace (funkcja zwróciła błąd), 2) jest zedecydowanie mniej wartści Score niż ROR (Score notowany jest w większym niż dzienny interwale).

In [49]:
rors_scores_df = pd.merge(df_rors_all, df[["Comp", "Date", "Score"]], how="left", on=["Comp", "Date"])
rors_scores_df

,Close,Comp,Date,ROR_month,ROR_qtr,ROR_half_year,ROR_year,Score
0,5.716019,UBA,2004-02-10,NaN,NaN,NaN,NaN,NaN
1,5.716019,UBA,2004-02-11,NaN,NaN,NaN,NaN,NaN
2,5.677787,UBA,2004-02-12,NaN,NaN,NaN,NaN,NaN
3,5.628081,UBA,2004-02-13,NaN,NaN,NaN,NaN,NaN
4,5.651024,UBA,2004-02-17,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5599544,123.379997,AMED,2022-05-19,-0.198096,NaN,-0.319647,-0.735906,NaN
5599545,126.400002,AMED,2022-05-20,-0.182453,NaN,NaN,-0.723842,NaN
5599546,121.150002,AMED,2022-05-23,NaN,-0.107504,-0.279830,NaN,NaN
5599547,117.209999,AMED,2022-05-24,NaN,-0.232772,-0.301535,-0.792475,NaN


In [50]:
rors_scores_df.isna().sum()

Close                 18
Comp                   0
Date                   0
ROR_month        2312063
ROR_qtr          1155117
ROR_half_year    1527215
ROR_year         1713116
Score            5569442
dtype: int64

In [52]:
comps_without_scores = []
for comp in rors_scores_df.Comp.unique():
    comp_df = rors_scores_df[rors_scores_df.Comp == comp]
    if comp_df.Score.isna().sum() == len(comp_df.Score):
        comps_without_scores.append(comp)

In [176]:
comps_without_scores

['MV', 'TEG', 'PGL', 'HOVVB', 'CMX', 'EE', 'CKR', 'POM', 'HET', 'WLMS', 'STR', 'NST', 'EMC', 'LDG', 'DRC', 'THI', 'GTK', 'ITC', 'NZ', 'ERT', 'PTV', 'NHP', 'HAR', 'MWP', 'BN', 'CEB', 'PRX', 'NVE', 'VCI', 'WRK', 'IM', 'CIXX', 'PETM', 'CLC', 'JH', 'PCP', 'IPMLF', 'DLM', 'NXG', 'FSH', 'HOT', 'PAS', 'AMRX', 'CLE', 'UIC', 'BEZ', 'MFE', 'PCL', 'BLC', 'KTO', 'PCZ', 'ONTO', 'MER', 'HMA', 'HUG', 'OMM', 'MDP', 'BDG.A', 'IDC']


## Usunięcie firm bez score'ów

In [53]:
rors_scores_clean = pd.DataFrame()
rors_scores_clean = rors_scores_df[~rors_scores_df['Comp'].isin(comps_without_scores)]

In [54]:
rors_scores_clean

,Close,Comp,Date,ROR_month,ROR_qtr,ROR_half_year,ROR_year,Score
0,5.716019,UBA,2004-02-10,NaN,NaN,NaN,NaN,NaN
1,5.716019,UBA,2004-02-11,NaN,NaN,NaN,NaN,NaN
2,5.677787,UBA,2004-02-12,NaN,NaN,NaN,NaN,NaN
3,5.628081,UBA,2004-02-13,NaN,NaN,NaN,NaN,NaN
4,5.651024,UBA,2004-02-17,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5599544,123.379997,AMED,2022-05-19,-0.198096,NaN,-0.319647,-0.735906,NaN
5599545,126.400002,AMED,2022-05-20,-0.182453,NaN,NaN,-0.723842,NaN
5599546,121.150002,AMED,2022-05-23,NaN,-0.107504,-0.279830,NaN,NaN
5599547,117.209999,AMED,2022-05-24,NaN,-0.232772,-0.301535,-0.792475,NaN


In [55]:
rors_scores_clean.shape

(5525581, 8)

In [56]:
rors_scores_clean.isna().sum()

Close                 18
Comp                   0
Date                   0
ROR_month        2279644
ROR_qtr          1136053
ROR_half_year    1501660
ROR_year         1682558
Score            5495474
dtype: int64

## Wyliczenie średnich stóp zwrotów

In [111]:
df_with_means = pd.DataFrame()
for comp in tqdm(rors_scores_clean.Comp.unique()):
    pom_df = pd.DataFrame()
    pom_df = rors_scores_clean[rors_scores_clean.Comp == comp].copy()
    score_index = pom_df[~pom_df.Score.isna()].index
    length += len(score_index)
    for indx in score_index:
        start_date = pom_df.loc[indx, 'Date']
        dates_indx = pom_df.loc[(pom_df.Date >= (start_date - timedelta(days = 7))) & (pom_df.Date <= start_date)].index
        pom_df.loc[indx, 'ROR_month'] = pom_df.loc[dates_indx ,'ROR_month'].mean()
        pom_df.loc[indx, 'ROR_qtr'] = pom_df.loc[dates_indx ,'ROR_qtr'].mean()
        pom_df.loc[indx, 'ROR_half_year'] = pom_df.loc[dates_indx ,'ROR_half_year'].mean()
        pom_df.loc[indx, 'ROR_year'] = pom_df.loc[dates_indx ,'ROR_year'].mean()
    
    df_with_means = pd.concat([df_with_means, pom_df[pom_df.index.isin(score_index)]])
df_with_means = df_with_means.reset_index(drop = True)
print(df_with_means)
    

100%|██████████| 1352/1352 [35:03<00:00,  1.56s/it]

            Close  Comp       Date  ROR_month   ROR_qtr  ROR_half_year  \
0        6.163362   UBA 2004-03-24   0.083411       NaN            NaN   
1        7.158419   UBA 2005-06-15   0.087692  0.098211       0.032070   
2        7.281594   UBA 2005-06-29   0.045381  0.134050       0.014216   
3        7.605220   UBA 2005-07-13   0.066313  0.228741       0.148893   
4        7.613519   UBA 2005-07-27   0.062501  0.249711       0.149382   
...           ...   ...        ...        ...       ...            ...   
30102   56.180000  AMED 2010-02-10   0.054901  0.305918       0.220840   
30103   58.970001  AMED 2017-07-12  -0.002406  0.152455       0.332011   
30104   48.689999  AMED 2017-07-26  -0.120590  0.079048       0.190007   
30105  122.250000  AMED 2019-03-13  -0.113183 -0.090947      -0.019737   
30106  119.559998  AMED 2019-03-27  -0.024715  0.099533       0.015505   

       ROR_year     Score  
0           NaN  0.741122  
1      0.254842  0.803913  
2      0.223035  0.806620  

In [183]:
df_with_means

,Close,Comp,Date,ROR_month,ROR_qtr,ROR_half_year,ROR_year,Score
0,6.163362,UBA,2004-03-24,0.083411,NaN,NaN,NaN,0.741122
1,7.158419,UBA,2005-06-15,0.087692,0.098211,0.032070,0.254842,0.803913
2,7.281594,UBA,2005-06-29,0.045381,0.134050,0.014216,0.223035,0.806620
3,7.605220,UBA,2005-07-13,0.066313,0.228741,0.148893,0.219617,0.923381
4,7.613519,UBA,2005-07-27,0.062501,0.249711,0.149382,0.305484,0.923270
...,...,...,...,...,...,...,...,...
30102,56.180000,AMED,2010-02-10,0.054901,0.305918,0.220840,0.206022,0.485908
30103,58.970001,AMED,2017-07-12,-0.002406,0.152455,0.332011,0.160072,0.689752
30104,48.689999,AMED,2017-07-26,-0.120590,0.079048,0.190007,0.075266,0.677474
30105,122.250000,AMED,2019-03-13,-0.113183,-0.090947,-0.019737,0.683718,0.751574


## Wczytanie danych dotyczących cen złota i oleju oraz połączenie z pozostałymi danymi

In [44]:
url = "https://raw.githubusercontent.com/MarylaSosna/umwf_projekt/main/oil_gold_yuan.csv"
df_finance_info = pd.read_csv(url, sep=",")
df_finance_info["Date"] = pd.to_datetime(df_finance_info["Date"])

In [60]:
df_means_oil_gold = pd.merge(df_with_means, df_finance_info, how="left", on=["Date"])
df_means_oil_gold

,Close,Comp,Date,ROR_month,ROR_qtr,ROR_half_year,ROR_year,Score,Oil,Gold,USD to Yuan
0,6.163362,UBA,2004-03-24,0.083411,NaN,NaN,NaN,0.741122,37.009998,417.200012,8.267201
1,7.158419,UBA,2005-06-15,0.087692,0.098211,0.032070,0.254842,0.803913,55.570000,429.100006,8.266501
2,7.281594,UBA,2005-06-29,0.045381,0.134050,0.014216,0.223035,0.806620,57.259998,NaN,8.266501
3,7.605220,UBA,2005-07-13,0.066313,0.228741,0.148893,0.219617,0.923381,60.009998,423.899994,8.266501
4,7.613519,UBA,2005-07-27,0.062501,0.249711,0.149382,0.305484,0.923270,59.110001,424.700012,8.102800
...,...,...,...,...,...,...,...,...,...,...,...
30102,56.180000,AMED,2010-02-10,0.054901,0.305918,0.220840,0.206022,0.485908,74.519997,1075.800049,6.819000
30103,58.970001,AMED,2017-07-12,-0.002406,0.152455,0.332011,0.160072,0.689752,45.490002,1218.099976,6.801700
30104,48.689999,AMED,2017-07-26,-0.120590,0.079048,0.190007,0.075266,0.677474,48.750000,1249.000000,6.749800
30105,122.250000,AMED,2019-03-13,-0.113183,-0.090947,-0.019737,0.683718,0.751574,58.259998,1307.500000,6.707600


In [61]:
df_all = pd.merge(df_means_oil_gold, df, how="left", on=["Comp", "Date", "Score"])

In [310]:
df_all

,Close,Comp,Date,ROR_month,ROR_qtr,ROR_half_year,ROR_year,Score,Oil,Gold,USD to Yuan,Category
0,6.163362,UBA,2004-03-24,0.083411,NaN,NaN,NaN,0.741122,37.009998,417.200012,8.267201,Finance
1,7.158419,UBA,2005-06-15,0.087692,0.098211,0.032070,0.254842,0.803913,55.570000,429.100006,8.266501,Finance
2,7.281594,UBA,2005-06-29,0.045381,0.134050,0.014216,0.223035,0.806620,57.259998,NaN,8.266501,Finance
3,7.605220,UBA,2005-07-13,0.066313,0.228741,0.148893,0.219617,0.923381,60.009998,423.899994,8.266501,Finance
4,7.613519,UBA,2005-07-27,0.062501,0.249711,0.149382,0.305484,0.923270,59.110001,424.700012,8.102800,Finance
...,...,...,...,...,...,...,...,...,...,...,...,...
30102,56.180000,AMED,2010-02-10,0.054901,0.305918,0.220840,0.206022,0.485908,74.519997,1075.800049,6.819000,Health Services
30103,58.970001,AMED,2017-07-12,-0.002406,0.152455,0.332011,0.160072,0.689752,45.490002,1218.099976,6.801700,Health Services
30104,48.689999,AMED,2017-07-26,-0.120590,0.079048,0.190007,0.075266,0.677474,48.750000,1249.000000,6.749800,Health Services
30105,122.250000,AMED,2019-03-13,-0.113183,-0.090947,-0.019737,0.683718,0.751574,58.259998,1307.500000,6.707600,Health Services


In [314]:
le = LabelEncoder()
df_all['Category'] = le.fit_transform(df_all['Category'])
df_all

,Close,Comp,Date,ROR_month,ROR_qtr,ROR_half_year,ROR_year,Score,Oil,Gold,USD to Yuan,Category
0,6.163362,UBA,2004-03-24,0.083411,NaN,NaN,NaN,0.741122,37.009998,417.200012,8.267201,8
1,7.158419,UBA,2005-06-15,0.087692,0.098211,0.032070,0.254842,0.803913,55.570000,429.100006,8.266501,8
2,7.281594,UBA,2005-06-29,0.045381,0.134050,0.014216,0.223035,0.806620,57.259998,NaN,8.266501,8
3,7.605220,UBA,2005-07-13,0.066313,0.228741,0.148893,0.219617,0.923381,60.009998,423.899994,8.266501,8
4,7.613519,UBA,2005-07-27,0.062501,0.249711,0.149382,0.305484,0.923270,59.110001,424.700012,8.102800,8
...,...,...,...,...,...,...,...,...,...,...,...,...
30102,56.180000,AMED,2010-02-10,0.054901,0.305918,0.220840,0.206022,0.485908,74.519997,1075.800049,6.819000,9
30103,58.970001,AMED,2017-07-12,-0.002406,0.152455,0.332011,0.160072,0.689752,45.490002,1218.099976,6.801700,9
30104,48.689999,AMED,2017-07-26,-0.120590,0.079048,0.190007,0.075266,0.677474,48.750000,1249.000000,6.749800,9
30105,122.250000,AMED,2019-03-13,-0.113183,-0.090947,-0.019737,0.683718,0.751574,58.259998,1307.500000,6.707600,9


In [315]:
# plik odkodowujący
filename = 'LabelEncoder.pickle'
pickle.dump(le, open(filename, 'wb'))

In [309]:
df_all.to_csv("df_all.csv")